[View in Colaboratory](https://colab.research.google.com/github/christianmerkwirth/colabs/blob/master/TPU_Resnet_Training_on_Imagenet_III.ipynb)

In [0]:
 # colab.research.google.com specific
import sys

if 'google.colab' in sys.modules:
  import json
  import os
  from google.colab import auth

  !git clone https://github.com/tensorflow/tpu.git
  !mv tpu/models/official .

In [0]:
# Now run the main to set up flags and constant. The main function should not be executed.

from official.resnet import resnet_main

In [0]:
# This cell contains the Colab-specific setup and hacks.

import json
import os
import sys
import time

from absl import flags
import absl.logging as _logging

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

FLAGS = flags.FLAGS

if 'google.colab' in sys.modules:
  from google.colab import auth

  # Authenticate to access GCS bucket
  auth.authenticate_user()

  # Initiate fake FLAGS parsing.
  FLAGS(['resnet_trainer'])
  
  # When connected to the TPU runtime
  if 'COLAB_TPU_ADDR' in os.environ:
    tpu_grpc = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

    FLAGS.tpu = tpu_grpc
    FLAGS.use_tpu = True

    # Upload credentials to the TPU
    with tf.Session(tpu_grpc) as sess:
      data = json.load(open('/content/adc.json'))
      tf.contrib.cloud.configure_gcs(sess, credentials=data)


In [0]:
from official.resnet import imagenet_input
from official.resnet import lars_util
from official.resnet import resnet_model


from tensorflow.python.estimator import estimator
      
# Number of training and evaluation images in the standard ImageNet dataset
NUM_TRAIN_IMAGES = 1281167
NUM_EVAL_IMAGES = 50000    

# Change parameters according to your setup.
FLAGS.model_dir = 'gs://tpu-cmerk-2/imagenet/models/resnet/20181022_008'
FLAGS.data_dir = 'gs://tpu-cmerk-2/imagenet/train'


# Define the learning rate schedule.
LR_SCHEDULE = [
      (0.001, 0), (1.0, 4), (0.1, 11), (0.2, 16), (0.05, 20), (0.02, 30), (0.01, 45), (0.05, 50), (0.001, 80)
]

def piecewise_linear_lr_schedule(current_epoch):
  """
  Piecewise linear learning rate.
  Args:
    current_epoch: `Tensor` for current epoch.
  Returns:
    A scaled `Tensor` for current learning rate.
  """
  scaled_lr = FLAGS.base_learning_rate * (FLAGS.train_batch_size / 256.0)
  lr = 0.0
  for i in range(1, len(LR_SCHEDULE)):
    this_lr = scaled_lr * ((LR_SCHEDULE[i][0] - LR_SCHEDULE[i-1][0]) /
                           (LR_SCHEDULE[i][1] - LR_SCHEDULE[i-1][1]) *
                           (current_epoch - LR_SCHEDULE[i-1][1]) +
                           LR_SCHEDULE[i-1][0])
    lr = tf.where(current_epoch >= LR_SCHEDULE[i-1][1], this_lr, lr)
  # If beyond last entry in schedule, keep last learning rate fixed.  
  lr = tf.where(current_epoch >= LR_SCHEDULE[-1][1], LR_SCHEDULE[-1][0], lr)
  return lr

# Override the original learning rate setter with this one.
resnet_main.learning_rate_schedule = piecewise_linear_lr_schedule


# Convert flags to dict of params so the model_fn can use TPU specific settings.
params = FLAGS.flag_values_dict()
    
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(FLAGS.tpu)

config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=FLAGS.model_dir,
    save_checkpoints_steps=FLAGS.iterations_per_loop,
    keep_checkpoint_max=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_cores,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

# Input pipelines are slightly different (with regards to shuffling and
# preprocessing) between training and evaluation.
imagenet_train = imagenet_input.ImageNetInput(
    is_training=True,
    data_dir=FLAGS.data_dir,
    use_bfloat16=True,
    transpose_input=FLAGS.transpose_input)
imagenet_eval = imagenet_input.ImageNetInput(
    is_training=False,
    data_dir=FLAGS.data_dir,
    use_bfloat16=True,
    transpose_input=FLAGS.transpose_input)

resnet_classifier = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=resnet_main.resnet_model_fn,
    config=config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    params=params)

In [0]:
current_step = estimator._load_global_step_from_checkpoint_dir(FLAGS.model_dir)
batches_per_epoch = NUM_TRAIN_IMAGES / FLAGS.train_batch_size
print('Training for %d steps (%.2f epochs in total). Current step %d.' % 
      (FLAGS.train_steps, FLAGS.train_steps / batches_per_epoch, current_step))

resnet_classifier.train(input_fn=imagenet_train.input_fn, max_steps=FLAGS.train_steps)

In [0]:
import time

eval_steps = NUM_EVAL_IMAGES // FLAGS.eval_batch_size

print('Starting to evaluate.')
eval_start = time.time()  # This time will include compilation time
eval_results = resnet_classifier.evaluate(
    input_fn=imagenet_eval.input_fn,
    steps=eval_steps)
eval_time = int(time.time() - eval_start)
print('Eval results: %s. Elapsed seconds: %d' % (eval_results, eval_time))